<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Electric-Power-Consumption-in-[GWh/a]" data-toc-modified-id="Electric-Power-Consumption-in-[GWh/a]-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Electric Power Consumption in [GWh/a]</a></span></li><li><span><a href="#Gas-consumption-in-[MWh/a]" data-toc-modified-id="Gas-consumption-in-[MWh/a]-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gas consumption in [MWh/a]</a></span></li></ul></div>

# Show the disaggregator overview

In [ ]:
from IPython.display import Image
pic = Image(filename=('./docs/_static/overview.png'))
pic

# Spatial disaggregation with `spatial.py`

Now, as we've gathered a spatial feeling for the relevant input data in the first notebook, let's take a look at the spatial disaggregation abilities.

In [ ]:
import pandas as pd
from disaggregator import config, data, spatial, plot

## Electric Power Consumption in [GWh/a]

The disaggregation method for the electric power consumption of private households is `spatial.disagg_households_power()` and requires at least one argument:

1. ``by`` is the distribution key and must be one of ``['population', 'households']``.  

    Using ``population`` invokes a *top-down*-method, i.e. the overall electricity consumption of entire Germany is being spread by distribution keys based on the population to calculate the consumption per region.  
    Using ``households`` invokes a *bottom-up*-method, i.e. the average electricity consumptions known for each household size are used for calculating the consumption per region.

Additionally, an income adjustment can be performed, through the switch ``weight_by_income=True``.

Let's start with ``population``:

In [ ]:
ec_pop = spatial.disagg_households_power(by='population', weight_by_income=False)
ec_pop.head()

... and continue with ``households``:

In [ ]:
ec_hh = spatial.disagg_households_power(by='households', weight_by_income=False)
ec_hh.head()

So, do these two fundamentally different methods lead to comparable results? Let's check, by summing the consumptions of all six household types per region:

In [ ]:
ec_hh.head().sum(axis=1)

Okay, that seems comparable.  
Let's further check the results by plotting these side by side on *one* map. We can achieve this, by merging the results into one new ``pd.DataFrame`` :

In [ ]:
ec_combined = pd.concat([ec_pop, ec_hh.sum(axis=1)], keys=['population', 'households'], axis=1, join='inner')
ec_combined.head()

... and plot:

In [ ]:
fig, ax = plot.choropleth_map(ec_combined, relative=True, unit='GWh', axtitle='Electricity consumption by', add_percentages=False)

Now let's weight the consumptions by the available income:

In [ ]:
ec_pop_ic = spatial.disagg_households_power(by='population', weight_by_income=True)
ec_hh_ic = spatial.disagg_households_power(by='households', weight_by_income=True)
ec_combined = pd.concat([ec_pop, ec_hh.sum(axis=1), ec_pop_ic, ec_hh_ic.sum(axis=1)],
                        keys=['population', 'households', 'population (weighted)', 'households (weighted)'],
                        axis=1, join='inner')

In [ ]:
fig, ax = plot.choropleth_map(ec_combined, relative=True, unit='GWh/a', axtitle='Electricity consumption by',
                              add_percentages=False, figsize=(20,12), nrows=1, ncols=4)

Now save this figure e.g. as PDF file:

In [ ]:
from disaggregator.config import data_out
fig.savefig(data_out('disagg_households_power.pdf'), bbox_inches='tight')

## Gas consumption in [MWh/a]

The gas demands of private households are threefold:
1. cooking (=process heat)
2. warm water
3. space heating

The first two ones depend mainly on the **household size**, whereas the latter depends mainly on the **size of the building**. However, centralized space heating systems are very often also used as boilers for the hot water supply.

In [ ]:
df_gas_td = spatial.disagg_households_gas(how='top-down')
df_gas_td.head()

In [ ]:
df_gas_bu = spatial.disagg_households_gas(how='bottom-up')
df_gas_bu.head()

In [ ]:
fig, ax = plot.choropleth_map(df_gas_td, unit='MWh_{gas}', colorbar_each_subplot=True, add_percentages=True, suptitle='Top-Down')

In [ ]:
fig, ax = plot.choropleth_map(df_gas_bu, unit='MWh_{gas}/a', colorbar_each_subplot=True, add_percentages=True, suptitle='Bottom-Up')